In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# 1. Load & Prepare Data
print("📂 Loading Integrated Data...")
try:
    df = pd.read_csv('chula_papers_with_quality.csv')
except FileNotFoundError:
    print("❌ Error: ไม่พบไฟล์ chula_papers_with_quality.csv (กรุณารัน Data Integration ก่อน)")
    exit()

df = df.dropna(subset=['SJR Best Quartile'])

df['target'] = df['SJR Best Quartile'].apply(lambda x: 1 if x == 'Q1' else 0)

def check_inter(country_str):
    country_str = str(country_str)
    if ';' in country_str: return 'Yes' 
    if 'Thailand' not in country_str: return 'Yes' 
    return 'No' 

df['is_inter'] = df['countries_str'].apply(check_inter)


df['primary_subject'] = df['subject_areas_str'].apply(lambda x: str(x).split(';')[0].strip())


X = df[['title', 'is_inter', 'primary_subject']]
y = df['target']

print(f"✅ Data Prepared: {len(df)} samples")
print(f"   - Q1 Papers: {y.sum()} ({y.sum()/len(y)*100:.1f}%)")
print(f"   - Other Papers: {len(y) - y.sum()}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2. Build AI Pipeline
print("\n🚀 Building & Training Model...")

preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(stop_words='english', max_features=3000, ngram_range=(1,2)), 'title'),

        ('cat', OneHotEncoder(handle_unknown='ignore'), ['is_inter', 'primary_subject'])
    ]
)

model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

model.fit(X_train, y_train)

# 3. Evaluation
print("\n📊 Evaluation Results:")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Q2-Q4', 'Q1']))

# 4. Save Model
joblib.dump(model, 'q1_predictor_model.joblib')
print("💾 Model saved to 'q1_predictor_model.joblib'")

unique_subjects = sorted(df['primary_subject'].unique())
joblib.dump(unique_subjects, 'subject_list.joblib')

📂 Loading Integrated Data...
✅ Data Prepared: 18537 samples
   - Q1 Papers: 9680 (52.2%)
   - Other Papers: 8857

🚀 Building & Training Model...

📊 Evaluation Results:
              precision    recall  f1-score   support

       Q2-Q4       0.68      0.69      0.69      1772
          Q1       0.71      0.70      0.71      1936

    accuracy                           0.70      3708
   macro avg       0.70      0.70      0.70      3708
weighted avg       0.70      0.70      0.70      3708

💾 Model saved to 'q1_predictor_model.joblib'


['subject_list.joblib']

In [ ]:
import pandas as pd
import joblib
import numpy as np

print("🚀 กำลังเจาะลึกสมอง AI เพื่อหาคำตอบ...")

try:
    model_pipeline = joblib.load('q1_predictor_model.joblib')
    df = pd.read_csv('chula_papers_with_quality.csv')
    df = df.dropna(subset=['SJR Best Quartile']) 
    df['is_Q1'] = df['SJR Best Quartile'].apply(lambda x: 1 if x == 'Q1' else 0)
    
    def check_inter(country_str):
        country_str = str(country_str)
        if ';' in country_str: return 'Yes'
        if 'Thailand' not in country_str: return 'Yes'
        return 'No'
    df['is_inter'] = df['countries_str'].apply(check_inter)
    
except Exception as e:
    print(f"❌ Error: {e}")
    exit()


print("\n" + "="*50)
print("1️⃣ ต้องมีเพื่อนต่างชาติช่วยเขียนด้วยไหม? (Collaboration Impact)")
print("="*50)

inter_q1_rate = df[df['is_inter'] == 'Yes']['is_Q1'].mean() * 100
local_q1_rate = df[df['is_inter'] == 'No']['is_Q1'].mean() * 100

print(f"🌍 งานที่มีต่างชาติร่วม (International):  โอกาสได้ Q1 คือ {inter_q1_rate:.2f}%")
print(f"🏠 งานที่มีแค่คนไทย (Local):           โอกาสได้ Q1 คือ {local_q1_rate:.2f}%")

if inter_q1_rate > local_q1_rate:
    diff = inter_q1_rate - local_q1_rate
    print(f"👉 สรุป: มีเพื่อนต่างชาติช่วย เพิ่มโอกาสได้ Q1 ถึง +{diff:.2f}%")
else:
    print("👉 สรุป: ไม่ต่างกันมาก")

print("\n" + "="*50)
print("2️⃣ ต้องตั้งชื่อเรื่องแบบไหน? (Top Keywords for Q1)")
print("="*50)

vectorizer = model_pipeline.named_steps['preprocessor'].transformers_[0][1]
feature_names = vectorizer.get_feature_names_out()

rf_model = model_pipeline.named_steps['classifier']
importances = rf_model.feature_importances_

text_importances = importances[:len(feature_names)]

indices = np.argsort(text_importances)[::-1]
top_n = 20

print("🔥 คำศัพท์ 20 คำแรก ที่มักจะปรากฏในงานวิจัย Q1 (ตามน้ำหนักความสำคัญ):")
for i in range(top_n):
    print(f"   {i+1}. {feature_names[indices[i]]} (Score: {text_importances[indices[i]]:.4f})")

print("\n💡 คำแนะนำ: ลองตั้งชื่อเรื่องโดยใช้คำเหล่านี้ หรือแนวทางนี้")

print("\n" + "="*50)
print("3️⃣ อยู่ที่คณะ/สาขาอะไร? (Top Subjects for Q1)")
print("="*50)

df['main_subject'] = df['subject_areas_str'].apply(lambda x: str(x).split(';')[0].strip())

subject_stats = df.groupby('main_subject')['is_Q1'].agg(['mean', 'count'])
subject_stats = subject_stats[subject_stats['count'] > 50] # เอาเฉพาะสาขาที่มีงานวิจัยเกิน 50 เรื่อง
subject_stats = subject_stats.sort_values(by='mean', ascending=False).head(10)

print("🏆 10 สาขาวิชาที่มีสัดส่วนงาน Q1 สูงที่สุด:")
for subject, row in subject_stats.iterrows():
    print(f"   - {subject}: {row['mean']*100:.2f}% (จาก {row['count']} เรื่อง)")

🚀 กำลังเจาะลึกสมอง AI เพื่อหาคำตอบ...

1️⃣ ต้องมีเพื่อนต่างชาติช่วยเขียนด้วยไหม? (Collaboration Impact)
🌍 งานที่มีต่างชาติร่วม (International):  โอกาสได้ Q1 คือ 62.76%
🏠 งานที่มีแค่คนไทย (Local):           โอกาสได้ Q1 คือ 42.16%
👉 สรุป: มีเพื่อนต่างชาติช่วย เพิ่มโอกาสได้ Q1 ถึง +20.60%

2️⃣ ต้องตั้งชื่อเรื่องแบบไหน? (Top Keywords for Q1)
🔥 คำศัพท์ 20 คำแรก ที่มักจะปรากฏในงานวิจัย Q1 (ตามน้ำหนักความสำคัญ):
   1. thailand (Score: 0.0063)
   2. using (Score: 0.0041)
   3. thai (Score: 0.0037)
   4. based (Score: 0.0034)
   5. analysis (Score: 0.0031)
   6. study (Score: 0.0031)
   7. new (Score: 0.0029)
   8. case (Score: 0.0026)
   9. production (Score: 0.0026)
   10. novel (Score: 0.0026)
   11. effect (Score: 0.0024)
   12. characterization (Score: 0.0023)
   13. effects (Score: 0.0023)
   14. patients (Score: 0.0023)
   15. high (Score: 0.0017)
   16. development (Score: 0.0017)
   17. review (Score: 0.0017)
   18. treatment (Score: 0.0016)
   19. non (Score: 0.0016)
   20. infection 